In [29]:
"""
CaboCha.FORMAT_LATTICEの出力形式

〈文頭が*の段〉
文節番号，かかり先の文節番号，主辞/機能語の位置，係り関係のスコア
(e.g.)* 0 17D 1/1 0.388993

〈文頭が*でない段〉
表層形，品詞，品詞細分類1，品詞細分類2，品詞細分類3，活用形，活用型，原型，読み，発音
(e.g.)人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
"""
import MeCab
tagger = MeCab.Tagger()  # 規定の辞書(unidic)を指定
basepath = "/Users/shirakawamomoko/Desktop/nlp100保存/chapter05/"

class Morph:
    def __init__(self, m):
        """
        m：形態素解析結果(e.g.)人工 名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
        """
        m = m.split('\t')  # ['人工', '名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー']
        m_features = m[1].split(",")  # ['名詞', '一般', '*', '*', '*', '*', '人工', 'ジンコウ', 'ジンコー\n']
        self.surface = m[0]
        self.base = m_features[6].replace("\n", "")  # カタカナ，ローマ字は出力が原型までしかないので，末尾に\nがつく
        self.pos = m_features[0]
        self.pos1 = m_features[1]

class Chunk:
    def __init__(self, morph_list, dst, srcs,pos_list,pos1_list,org_list):
        """
        morph_list：文節ごとの形態素のlist
        dst：係り先文節インデックス番号
        srcs：係り元文節インデックス番号(e.g.)"3/2/1"
        """
        srcs = srcs.split("/")
        self.morphs = morph_list
        self.dst = dst
        self.srcs = srcs
        self.pos = pos_list
        self.pos1 = pos1_list
        self.original = org_list
with open(basepath + "ai.ja.txt.parsed", "r") as f:
    lines = f.readlines()
    ai_phrase = []  # 最終的な保存先
    sentence_list = []  # 文章ごとの保存先
    morph_list = []  # sentence_listの形態素部分の保存先
    srcs_dict = {}  # 文節：係り受け先となる文節の辞書を作成
    org_list=[]#原形の保存

    for line in lines:
        line = line.split()  # ['人工', '名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー']
        if line[0].strip() == "EOS":
            if len(sentence_list) > 0:  # 1つでもlistに入っていれば
                ai_phrase.append(sentence_list)
            morph_list,pos_list,pos1_list,sentence_list,org_list= [],[],[],[],[]
            srcs_dict = {}
        elif line[0] == "*":
            dst = line[2].replace("D", "")
            # srcsの辞書を作成していく
            key = line[2].replace("D", "")
            value = line[1]
            try:
                srcs_dict[key] += "/" + value
            except:  # まだ辞書内にkeyが存在しなければ
                srcs_dict[key] = value
            morph_list, pos_list,pos1_list,org_list= [],[],[],[]
            sentence_list.append(Chunk(morph_list, dst, srcs_dict.get(line[1], "なし"),pos_list,pos1_list,org_list))#.get：キーがなかったら空白
        else:
            morph_list.append(Morph('\t'.join(line)))  # 形態素を追加
            pos_list.append(line[1].split(",")[0])
            pos1_list.append(line[1].split(",")[1])
            org_list.append(line[1].split(",")[6])
f.close()

# 文節リストに対してsrcs_dictを使って変換を行う
ai_phrase_dict = []
for sentence in ai_phrase:
    sentence_dict = []
    for chunk in sentence:
        chunk_dict = {
            "morphs": [morph.surface for morph in chunk.morphs],
            "dst": chunk.dst,
            "srcs": chunk.srcs,
            "pos" : chunk.pos,
            "pos1" : chunk.pos1,
            "original" : chunk.original
        }
        sentence_dict.append(chunk_dict)
    ai_phrase_dict.append(sentence_dict)

# ai_phrase_dictの内容を表示する
"""
sentence = ai_phrase_dict[1]
for chunk in sentence:
    print("morphs(形態素のリスト)：",chunk["morphs"])
    print("dst(係り先文節インデックス番号)：",chunk['dst'])
    print("srcs(係り元文節インデックス番号)：",chunk['srcs'])
    print(chunk["pos1"])
    print("-----")
"""


morphs(形態素のリスト)： ['人工', '知能']
dst(係り先文節インデックス番号)： 17
srcs(係り元文節インデックス番号)： ['なし']
['一般', '一般']
-----
morphs(形態素のリスト)： ['（', 'じん', 'こうち', 'のう', '、', '、']
dst(係り先文節インデックス番号)： 17
srcs(係り元文節インデックス番号)： ['なし']
['括弧開', '一般', '一般', '終助詞', '読点', '読点']
-----
morphs(形態素のリスト)： ['AI']
dst(係り先文節インデックス番号)： 3
srcs(係り元文節インデックス番号)： ['なし']
['一般']
-----
morphs(形態素のリスト)： ['〈', 'エーアイ', '〉', '）', 'と', 'は', '、']
dst(係り先文節インデックス番号)： 17
srcs(係り元文節インデックス番号)： ['2']
['括弧開', '固有名詞', '括弧閉', '括弧閉', '格助詞', '係助詞', '読点']
-----
morphs(形態素のリスト)： ['「', '『', '計算']
dst(係り先文節インデックス番号)： 5
srcs(係り元文節インデックス番号)： ['なし']
['括弧開', '括弧開', 'サ変接続']
-----
morphs(形態素のリスト)： ['（', '）', '』', 'という']
dst(係り先文節インデックス番号)： 9
srcs(係り元文節インデックス番号)： ['4']
['括弧開', '括弧閉', '括弧閉', '格助詞']
-----
morphs(形態素のリスト)： ['概念', 'と']
dst(係り先文節インデックス番号)： 9
srcs(係り元文節インデックス番号)： ['なし']
['一般', '並立助詞']
-----
morphs(形態素のリスト)： ['『', 'コンピュータ']
dst(係り先文節インデックス番号)： 8
srcs(係り元文節インデックス番号)： ['なし']
['括弧開', '一般']
-----
morphs(形態素のリスト)： ['（', '）', '』', 'という']
dst(係り先文節インデックス番号)： 9
s